In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from classification import *
from numpy import genfromtxt
import pywt
from kernel import *
from datetime import time, datetime, timedelta

In [3]:
import cvxopt

In [4]:
Xtr = genfromtxt('../data/Xtr.csv', delimiter=',')
Ytr = genfromtxt('../data/Ytr.csv', delimiter=',')
Xte = genfromtxt('../data/Xte.csv', delimiter=',')

Xtr = np.delete(Xtr, 3072, axis=1)
Xte = np.delete(Xte, 3072, axis=1)
Ytr = Ytr[1:,1]
N = len(Ytr)

In [5]:
def wavelet_transform(Xtr):
    result = np.zeros((Xtr.shape[0], 972))

    for i in range(Xtr.shape[0]):
        r = Xtr[i][:1024].reshape(32,32)
        g = Xtr[i][1024:2048].reshape(32,32)
        b = Xtr[i][-1024:].reshape(32,32)

        rgbArray = np.zeros((32,32,3), 'uint8')
        rgbArray[..., 0] = (r+r.min())/(r.max()-r.min())*256
        rgbArray[..., 1] = (g+g.min())/(g.max()-g.min())*256
        rgbArray[..., 2] = (b+b.min())/(b.max()-b.min())*256

        grayImage = 0.2989 *rgbArray[..., 0]+ 0.5870 *rgbArray[..., 1]+0.1140 *rgbArray[..., 2]
        coeffs2 = pywt.dwt2(grayImage, 'bior1.3')
        LL, (LH, HL, HH) = coeffs2

        result[i,:] = np.concatenate((LH.ravel(), HL.ravel(), HH.ravel()), axis=0)
    return result

In [6]:
def one_versus_all_SVM(features, labels, _lambda):
    N = len(labels)
    n_labels = len(set(labels))
    alphas = np.zeros((n_labels, N))
    bias = np.zeros(n_labels)
    
    #Linear Kernel:
    K = features.T.dot(features)
    
    for label in range(n_labels):
        one_versus_all_labels = np.zeros(N)
        for i in range(N):
            if labels[i] == label:
                one_versus_all_labels[i] = 1
            else:
                one_versus_all_labels[i] = -1
        alphas[label, :], bias[label] = train_SVM(K, one_versus_all_labels, _lambda)
        print "classifier for label ", label, " done"
        
    return alphas, bias

def predict_SVM(alphas, bias, features, X):    
    y_pred = np.zeros(alphas.shape[0])
    values_pred = np.zeros((alphas.shape[0],X.shape[1]))
    for k in range(alphas.shape[0]):
        values_pred[k,:] = alphas[k,:].dot(features.T.dot(X))+bias[k]
    return np.argmax(values_pred, axis=0)

def train_SVM(K, y, _lambda):
    
    n = y.shape[0]
    gamma = 1 / (2 * _lambda * n)
    
    P = cvxopt.matrix(K)
    
    h = cvxopt.matrix(0., (2 * n, 1))
    h[:n] = gamma
    
    A = cvxopt.matrix(1., (1, n))
    b = cvxopt.matrix(0.)
    
    y = y.astype(np.double)
    diag_y = cvxopt.spdiag(y.tolist())
    q = cvxopt.matrix(-y)
    G = cvxopt.sparse([diag_y, -diag_y])    

    res = cvxopt.solvers.qp(P, q, G, h, A, b)
    
    return np.array(res["x"]).T, res["y"][0]



## Test new SVM

In [7]:
Xtr_t = fourier_modulus_2D_kernel(Xtr)

In [8]:
Xtr_train = Xtr_t[:4000, :].T
Xtr_test = Xtr_t[4000:, :].T
Ytr_train = Ytr[:4000]
Ytr_test = Ytr[4000:]

In [9]:
features = Xtr_train
labels = Ytr_train
X = Xtr_test

In [10]:
t1 = datetime.now()
alphas, bias = one_versus_all_SVM(features, labels, _lambda=0.1)
print 'model fitted'
prediction = predict_SVM(alphas, bias, features, X)
t2 = datetime.now()
print t2-t1

     pcost       dcost       gap    pres   dres
 0: -4.2241e+02 -1.1445e+01  3e+04  2e+02  5e-11
 1: -8.2988e+00 -1.1343e+01  5e+02  3e+00  5e-11
 2: -1.5322e+00 -1.0160e+01  4e+01  2e-01  3e-12
 3: -9.5904e-01 -5.6220e+00  7e+00  2e-02  4e-13
 4: -8.6546e-01 -2.1499e+00  2e+00  4e-03  1e-13
 5: -8.9988e-01 -1.3425e+00  5e-01  9e-04  8e-14
 6: -9.3449e-01 -1.1001e+00  2e-01  2e-04  8e-14
 7: -9.5699e-01 -1.0077e+00  5e-02  3e-06  9e-14
 8: -9.6705e-01 -9.8321e-01  2e-02  6e-09  1e-13
 9: -9.7062e-01 -9.7573e-01  5e-03  5e-17  1e-13
10: -9.7218e-01 -9.7308e-01  9e-04  1e-17  9e-14
11: -9.7251e-01 -9.7257e-01  6e-05  3e-17  9e-14
12: -9.7253e-01 -9.7253e-01  2e-06  2e-17  9e-14
13: -9.7253e-01 -9.7253e-01  4e-08  5e-17  1e-13
Optimal solution found.
classifier for label  0  done
     pcost       dcost       gap    pres   dres
 0: -3.5701e+02 -1.1491e+01  3e+04  2e+02  4e-11
 1: -6.9179e+00 -1.1386e+01  5e+02  3e+00  5e-11
 2: -1.4415e+00 -1.0135e+01  4e+01  2e-01  3e-12
 3: -9.1409e-01 -

In [11]:
well_classified = 0
for i in range(len(prediction)):
    if prediction[i] == Ytr_test[i]:
         well_classified+=1
print float(well_classified)/len(Ytr_test)

0.32


## Toy data for tests

In [ ]:
n_samples = 150
mean_1 = [0, -3]
cov = [[3, 0], [0, 3]]
X_1 = np.random.multivariate_normal(mean_1, cov, n_samples).T
mean_2 = [3, 3]
X_2 = np.random.multivariate_normal(mean_2, cov, n_samples).T
mean_3 = [-3, 3]
X_3 = np.random.multivariate_normal(mean_3, cov, n_samples).T
X = np.concatenate((X_1, X_2, X_3), axis = 1)

y = np.concatenate((np.zeros((1,n_samples)), np.ones((1, n_samples)), 2*np.ones((1,n_samples))), axis=1)
y = y[0,:]
Xtr_t = X.T
Ytr = y

mask_test = range(0,450, 5)
mask_train = [i for i in range(450) if i not in mask_test]

Xtr_train = Xtr_t[mask_train, :].T
Xtr_test = Xtr_t[mask_test, :].T
Ytr_train = Ytr[mask_train]
Ytr_test = Ytr[mask_test]